In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
data=pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
data.head()

In [34]:
data.describe()

In [38]:

{x:len(data[x].unique()) for x in data.columns}

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression

def mi_score(X,y):
    mi=mutual_info_regression(X,y)
    mi=pd.Series(mi,name='MI Scores',index=X.columns)
    mi=mi.sort_values(ascending=False)
    return mi
y=data['target']
X=data.drop(['target'],axis=1)
scores=mi_score(X,y)
print(scores)

#sns.distplot(x='age',y='target',data=data)

In [7]:
import math
import numpy as np
cols_to_use=['thal','slope','cp','oldpeak','exang','feature1','ca','thalach','chol','target']
data['feature1']=np.log(data['chol']+0.001)*(data['thalach']+0.2)
X['feature1']=data['feature1']
sns.lmplot(x='feature1',y='chol',hue='target',data=data)
sns.swarmplot(x='target',y='feature1',data=data)

In [35]:
sns.distplot(data['feature1'],kde=False)
sns.distplot(data[data['target']==1]['feature1'],kde=False)

In [33]:
numHeartDisease=data['target'].sum()
labels=['Yes','No']
ax=plt.pie([numHeartDisease,len(data)-numHeartDisease],labels=labels,autopct='%1.1f%%',shadow=True,explode=(0.1,0))
plt.show

In [39]:
sns.distplot(data['chol'],kde=False)
sns.distplot(data[data['target']==1]['chol'],kde=False)

In [6]:
sns.distplot(data['chol'])
sns.distplot(data.loc[data['target']==1]['chol'])

In [7]:
cols=[x for x in X.columns]
print(cols)

In [8]:
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=8)
X['Cluster']=kmeans.fit_predict(X)
X['Cluster']=X['Cluster'].astype("category")
X.head()

In [9]:
sns.distplot(X['Cluster'],kde=False)
sns.distplot(X.loc[data['target']==1]['Cluster'],kde=False)

In [10]:
sns.lmplot(x="thalach",y="chol",hue="cp",data=data,height=6)

In [11]:
from sklearn.model_selection import train_test_split
X=X[X.columns.intersection(cols_to_use)]
X_train,X_valid,y_train,y_valid=train_test_split(X,y)
print(X_train.shape)

In [60]:
from xgboost import XGBRegressor

my_model=XGBRegressor(n_estimators=1000,learning_rate=0.01)
my_model.fit(X_train,y_train)


In [61]:
preds=my_model.predict(X_valid)
preds=[0 if x<=0.5 else 1 for x in preds]

In [62]:
from sklearn.metrics import mean_absolute_error,accuracy_score, confusion_matrix
print(accuracy_score(preds,y_valid))

In [63]:
from sklearn import svm
svm_model=svm.SVC()
svm_model.fit(X_train,y_train)

In [64]:
svm_preds=svm_model.predict(X_valid)
preds=[0 if x<0.5 else 1 for x in svm_preds]
print(accuracy_score(preds,y_valid))

In [66]:
from sklearn.tree import DecisionTreeClassifier
tree_model=DecisionTreeClassifier()
tree_model.fit(X_train,y_train)
tree_preds=tree_model.predict(X_valid)
tree_preds=[0 if x<0.5 else 1 for x in tree_preds]
print(accuracy_score(tree_preds,y_valid))

In [12]:
from keras.models import Sequential
from keras.layers import Dense

keras_model=Sequential()
keras_model.add(Dense(12,input_dim=9,activation='relu'))
keras_model.add(Dense(4,activation='relu'))
keras_model.add(Dense(1,activation='sigmoid'))

keras_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
keras_model.fit(X_train,y_train,epochs=1000)

In [13]:
_,accuracy=keras_model.evaluate(X_valid,y_valid)
print(accuracy)

In [21]:
keras_model.save('./')

In [26]:
from tensorflow import keras
p=keras.models.load_model('./')

In [29]:
_,accuracy=p.evaluate(X_valid,y_valid)
print(accuracy)

In [17]:
import pickle
with open('heart_model','wb') as files:
    pickle.dump(keras_model,files)

In [30]:
!zip -r file.zip './'